[View in Colaboratory](https://colab.research.google.com/github/shinchan75034/testrep/blob/master/NMT1.ipynb)

In [2]:
import numpy as np
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [0]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

In [0]:
# load doc into memory
def load_doc(filename):
    import io
    # open the file as read only
    file = io.open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [0]:
# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in lines]
    return pairs

In [0]:
# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [re_punc.sub('', w) for w in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

In [0]:
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [13]:
# Download the file.
!gsutil cp gs://bucket-kctung75034-1/deu.txt /tmp/gsutil_download.txt


Copying gs://bucket-kctung75034-1/deu.txt...
/ [1 files][ 11.8 MiB/ 11.8 MiB]                                                
Operation completed over 1 objects/11.8 MiB.                                     


In [0]:
# load dataset
filename = '/tmp/gsutil_download.txt'
doc = load_doc(filename)
# split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)

In [15]:
clean_pairs

array([['hi', 'hallo'],
       ['hi', 'gru gott'],
       ['run', 'lauf'],
       ...,
       ['at a moment when our economy is growing our businesses are creating jobs at the fastest pace since the and wages are starting to rise again we have to make some choices about the kind of country we want to be',
        'in einem moment in dem unsere wirtschaft wachst unsere betriebe so schnell neue arbeitsplatze schaffen wie zuletzt in den und die gehalter steigen mussen wir entscheidungen treffen und uns uberlegen was fur ein land wir sein wollen'],
       ['if someone who doesnt know your background says that you sound like a native speaker it means they probably noticed something about your speaking that made them realize you werent a native speaker in other words you dont really sound like a native speaker',
        'wenn jemand der deine herkunft nicht kennt sagt dass du wie ein muttersprachler sprichst bedeutet das dass man wahrscheinlich etwas an deiner sprechweise bemerkt hat das erk

In [17]:
clean_pairs.shape

numpy.ndarray

In [0]:
from tensorflow.python.lib.io import file_io
np.save(file_io.FileIO('gs://bucket-kctung75034-1/english-german.pkl', 'w'), clean_pairs)